# RFM SEGMENTATION

# Introduction
**RFM segmentation** is a marketing analysis technique that classifies customers based on their recent purchasing behavior, the frequency of their purchases, and the monetary value of those purchases. The acronym RFM stands for:

The RFM segmentation process involves assigning each customer a score for each of these three factors based on their historical transaction data. Scores are usually assigned on a scale of 1 to 5, with higher scores indicating higher recency, frequency, or monetary value.

Once the scores are assigned, customers can be grouped into segments based on their RFM scores. For example, a customer with high scores for recency, frequency, and monetary value might be in the "VIP" segment, while a customer with lower scores in these categories might be in the "Needs Attention" segment.

In [2]:
import pandas as pd
import numpy as np
import datetime as dt
import matplotlib.pyplot as plt
import seaborn as sns
import plotly.express as px

In [20]:
df = pd.read_csv('Online_Retail_Data.csv', header=0)

In [21]:
df.head()

,order_id,product_code,product_name,quantity,order_date,price,customer_id
0,493410,TEST001,This is a test product.,5,2010-01-04 09:24:00,4.50,12346.0
1,C493411,21539,RETRO SPOTS BUTTER DISH,-1,2010-01-04 09:43:00,4.25,14590.0
2,493412,TEST001,This is a test product.,5,2010-01-04 09:53:00,4.50,12346.0
3,493413,21724,PANDA AND BUNNIES STICKER SHEET,1,2010-01-04 09:54:00,0.85,NaN
4,493413,84578,ELEPHANT TOY WITH BLUE T-SHIRT,1,2010-01-04 09:54:00,3.75,NaN


In [22]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 461773 entries, 0 to 461772
Data columns (total 7 columns):
 #   Column        Non-Null Count   Dtype  
---  ------        --------------   -----  
 0   order_id      461773 non-null  object 
 1   product_code  461773 non-null  object 
 2   product_name  459055 non-null  object 
 3   quantity      461773 non-null  int64  
 4   order_date    461773 non-null  object 
 5   price         461773 non-null  float64
 6   customer_id   360853 non-null  float64
dtypes: float64(2), int64(1), object(4)
memory usage: 24.7+ MB


In [23]:
data = df.copy()

## Data Cleansing

In [24]:
# Convertion order date format to datetime
data['order_date'] = pd.to_datetime(data['order_date']).dt.date.astype('datetime64')

In [25]:
data.isnull().sum()

order_id             0
product_code         0
product_name      2718
quantity             0
order_date           0
price                0
customer_id     100920
dtype: int64

In [26]:
# Deleted customer id null

data = data[~ data['customer_id'].isna()]

In [27]:
data.isnull().sum()

order_id        0
product_code    0
product_name    0
quantity        0
order_date      0
price           0
customer_id     0
dtype: int64

In [28]:
data.duplicated().sum()

6411

In [29]:
# Deleted Duplicated data
data = data.drop_duplicates()

In [30]:
# Make product name lower
data['product_name'] = data['product_name'].str.lower()

In [31]:
# Make quantity all positive
data['quantity'] = data['quantity'].abs()

In [32]:
# Make new colomn for order_id cenceled
data['order_status'] = np.where(data['order_id'].str[:1] == 'C', 'cenceled', 'delivered')

In [33]:
data = data[data['price']>0]

In [34]:
# Make column amount
data['amount'] = data['quantity']*data['price']

In [35]:
# Deleted outlier
from scipy import stats

data = data[(np.abs(stats.zscore(data[['quantity', 'amount']]))<3).all(axis=1)]
data = data.reset_index(drop=True)

In [36]:
data.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 352075 entries, 0 to 352074
Data columns (total 9 columns):
 #   Column        Non-Null Count   Dtype         
---  ------        --------------   -----         
 0   order_id      352075 non-null  object        
 1   product_code  352075 non-null  object        
 2   product_name  352075 non-null  object        
 3   quantity      352075 non-null  int64         
 4   order_date    352075 non-null  datetime64[ns]
 5   price         352075 non-null  float64       
 6   customer_id   352075 non-null  float64       
 7   order_status  352075 non-null  object        
 8   amount        352075 non-null  float64       
dtypes: datetime64[ns](1), float64(3), int64(1), object(4)
memory usage: 24.2+ MB


In [37]:
# Convertion customer_id to string
data['customer_id'] = data['customer_id'].astype(str)

## MAKING RFM SEGMENTATION

In [38]:
df_user = data.copy()

In [39]:
df_user = df_user.groupby('customer_id', as_index = False).agg(order_count =('order_id', 'nunique'),
 max_order_date=('order_date', 'max'), order_value=('amount', 'sum'))

df_user.head()

,customer_id,order_count,max_order_date,order_value
0,12346.0,10,2010-10-04,683.96
1,12608.0,1,2010-10-31,415.79
2,12745.0,2,2010-08-10,723.85
3,12746.0,2,2010-06-30,266.35
4,12747.0,19,2010-12-13,4075.29


In [40]:
# Delta the day from the last day order

today = df_user['max_order_date'].max()
df_user['Last_day_order'] = (today - df_user['max_order_date']).dt.days
df_user.head()

,customer_id,order_count,max_order_date,order_value,Last_day_order
0,12346.0,10,2010-10-04,683.96,80
1,12608.0,1,2010-10-31,415.79,53
2,12745.0,2,2010-08-10,723.85,135
3,12746.0,2,2010-06-30,266.35,176
4,12747.0,19,2010-12-13,4075.29,10


In [41]:
df_user.describe()

,order_count,order_value,Last_day_order
count,3890.000000,3890.000000,3890.000000
mean,5.129820,1536.724008,90.751671
std,8.499322,3430.178165,88.820754
min,1.000000,1.250000,0.000000
25%,1.000000,294.132500,25.000000
50%,3.000000,645.925000,58.000000
75%,6.000000,1580.427500,126.000000
max,163.000000,71970.390000,352.000000


In [42]:
# Grouping Recency Score
df_user['recency_score'] = pd.cut(df_user['Last_day_order'],
                        bins = [df_user['Last_day_order'].min(),
                        np.percentile(df_user['Last_day_order'], 20),
                        np.percentile(df_user['Last_day_order'], 40),
                        np.percentile(df_user['Last_day_order'], 60),
                        np.percentile(df_user['Last_day_order'], 80),
                        df_user['Last_day_order'].max()],
                        labels=[5,4,3,2,1],
                        include_lowest = True).astype(int)

In [43]:
# Grouping frequency score
df_user['frequency_score'] = pd.cut(df_user['order_count'],
                        bins = [0,
                        np.percentile(df_user['order_count'], 20),
                        np.percentile(df_user['order_count'], 40),
                        np.percentile(df_user['order_count'], 60),
                        np.percentile(df_user['order_count'], 80),
                        df_user['order_count'].max()],
                        labels=[1,2,3,4,5],
                        include_lowest = True).astype(int)
df_user.head()

,customer_id,order_count,max_order_date,order_value,Last_day_order,recency_score,frequency_score
0,12346.0,10,2010-10-04,683.96,80,2,5
1,12608.0,1,2010-10-31,415.79,53,3,1
2,12745.0,2,2010-08-10,723.85,135,2,2
3,12746.0,2,2010-06-30,266.35,176,1,2
4,12747.0,19,2010-12-13,4075.29,10,5,5


In [44]:
# Grouping monetary score
df_user['monetary_score'] = pd.cut(df_user['order_value'],
                        bins = [df_user['order_value'].min(),
                        np.percentile(df_user['order_value'], 20),
                        np.percentile(df_user['order_value'], 40),
                        np.percentile(df_user['order_value'], 60),
                        np.percentile(df_user['order_value'], 80),
                        df_user['order_value'].max()],
                        labels=[1,2,3,4,5],
                        include_lowest = True).astype(int)
df_user.head()

,customer_id,order_count,max_order_date,order_value,Last_day_order,recency_score,frequency_score,monetary_score
0,12346.0,10,2010-10-04,683.96,80,2,5,3
1,12608.0,1,2010-10-31,415.79,53,3,1,2
2,12745.0,2,2010-08-10,723.85,135,2,2,3
3,12746.0,2,2010-06-30,266.35,176,1,2,2
4,12747.0,19,2010-12-13,4075.29,10,5,5,5


In [45]:
# Segmentation Frequency vs Recency

df_user['segment'] = np.select(
    [(df_user['recency_score']==5) & (df_user['frequency_score']>=4),
     (df_user['recency_score'].between(3, 4)) & (df_user['frequency_score']>=4),
     (df_user['recency_score']>=4) & (df_user['frequency_score'].between(2, 3)),
     (df_user['recency_score']<=2) & (df_user['frequency_score']==5),
     (df_user['recency_score']==3) & (df_user['frequency_score']==3),
     (df_user['recency_score']==5) & (df_user['frequency_score']==1),
     (df_user['recency_score']==4) & (df_user['frequency_score']==1),
     (df_user['recency_score']<=2) & (df_user['frequency_score'].between(3, 4)),
     (df_user['recency_score']==3) & (df_user['frequency_score']<=2),
     (df_user['recency_score']<=2) & (df_user['frequency_score']<=2)],
    ['01-Champion', '02-Loyal Customers', '03-Potential Loyalists', "04-Can't Lose Them", '05-Need Attention',
     '06-New Customers', '07-Promising', '08-At Risk', '09-About to Sleep', '10-Hibernating']
)
df_user.head()

,customer_id,order_count,max_order_date,order_value,Last_day_order,recency_score,frequency_score,monetary_score,segment
0,12346.0,10,2010-10-04,683.96,80,2,5,3,04-Can't Lose Them
1,12608.0,1,2010-10-31,415.79,53,3,1,2,09-About to Sleep
2,12745.0,2,2010-08-10,723.85,135,2,2,3,10-Hibernating
3,12746.0,2,2010-06-30,266.35,176,1,2,2,10-Hibernating
4,12747.0,19,2010-12-13,4075.29,10,5,5,5,01-Champion


In [46]:
summary = pd.pivot_table(df_user, index='segment',
               values=['customer_id','Last_day_order','order_count','order_value'],
               aggfunc={'customer_id': pd.Series.nunique,
                        'Last_day_order': [np.mean, np.median],
                        'order_count': [np.mean, np.median],
                        'order_value': [np.mean, np.median]})
summary['pct_unique'] = (summary['customer_id'] / summary['customer_id'].sum() * 100).round(1)
summary

Last_day_order        customer_id order_count         \
                                 mean median     nunique        mean median   
segment                                                                       
01-Champion                 10.618182    9.5         550   15.469091   10.0   
02-Loyal Customers          40.864469   37.0         546    8.767399    7.0   
03-Potential Loyalists      23.573614   24.0         523    2.829828    3.0   
04-Can't Lose Them         121.338462  112.0          65   11.353846   10.0   
05-Need Attention           58.613636   59.0         176    3.397727    3.0   
06-New Customers            14.220000   16.0          50    1.000000    1.0   
07-Promising                32.760563   34.0         142    1.000000    1.0   
08-At Risk                 140.455399  120.0         426    4.136150    4.0   
09-About to Sleep           58.735795   58.0         352    1.417614    1.0   
10-Hibernating             196.667925  198.5        1060    1.312264    1.0   

                        order_value           pct_unique  
                               mean    median             
segment                                                   
01-Champion             4984.731500  2768.465       14.1  
02-Loyal Customers      2610.442570  1932.275       14.0  
03-Potential Loyalists   760.466577   613.690       13.4  
04-Can't Lose Them      2797.351538  2248.770        1.7  
05-Need Attention        982.986483   825.605        4.5  
06-New Customers         240.044400   183.790        1.3  
07-Promising             283.935070   233.645        3.7  
08-At Risk              1146.382819   860.980       11.0  
09-About to Sleep        443.948864   330.385        9.0  
10-Hibernating           340.985710   256.650       27.2

In [47]:
df1 = df_user.copy()


In [56]:
df2 = df1.groupby(['segment', 'customer_id'], as_index=False).agg(unique_user=('customer_id','nunique'))
df2

,segment,customer_id,unique_user
0,01-Champion,12747.0,1
1,01-Champion,12748.0,1
2,01-Champion,12826.0,1
3,01-Champion,12839.0,1
4,01-Champion,12841.0,1
...,...,...,...
3885,10-Hibernating,18279.0,1
3886,10-Hibernating,18281.0,1
3887,10-Hibernating,18284.0,1
3888,10-Hibernating,18285.0,1


In [60]:
from os import name
values = df2 = df2.groupby(['segment'])['unique_user'].sum().reset_index(name ='Unique user')
values

,segment,Unique user
0,01-Champion,550
1,02-Loyal Customers,546
2,03-Potential Loyalists,523
3,04-Can't Lose Them,65
4,05-Need Attention,176
5,06-New Customers,50
6,07-Promising,142
7,08-At Risk,426
8,09-About to Sleep,352
9,10-Hibernating,1060


In [72]:
# Segmentation Visualisasi
df2 = df1.groupby(['segment', 'customer_id'], as_index=False).agg(unique_user=('customer_id','nunique'))
df2 = df2 = df2.groupby(['segment'])['unique_user'].sum().reset_index(name ='unique_user')
fig = px.bar(data_frame=df2, x='segment', y='unique_user', height=400, width=500, color_discrete_sequence=['blue'],
             title='RFM Segmentattion')
fig.show()

From the graph, we know that '10-Hibernating' users have the biggest values.

In [64]:
# Recancy Score Visualisai
df3 = df1.groupby(['recency_score', 'customer_id'], as_index=False).agg(unique_user=('customer_id','nunique'))
df3 = df3.groupby(['recency_score'])['unique_user'].sum().reset_index(name ='unique_user')
fig = px.bar(data_frame=df3, x='recency_score', y='unique_user', height=400, width=500, color_discrete_sequence=['blue'], title='Recency Score')
fig.show()

From score 1 until 5, the recency score almost has the same values.

In [66]:
# Frequency Score Visualisasi
df4 = df1.groupby(['frequency_score', 'customer_id'], as_index=False).agg(unique_user=('customer_id','nunique'))
df4 = df4.groupby(['frequency_score'])['unique_user'].sum().reset_index(name ='unique_user')
fig = px.bar(data_frame=df4, x='frequency_score', y='unique_user', height=300, width=500, color_discrete_sequence=['blue'], title='Frequency Score')
fig.show()

Frequency score 1 has the biggest value among the others. This means that the frequency of user transactions is low.

In [67]:
# Monetary Score Visualisasi
df5 = df1.groupby(['monetary_score', 'customer_id'], as_index=False).agg(unique_user=('customer_id','nunique'))
df5 = df5.groupby(['monetary_score'])['unique_user'].sum().reset_index(name='unique_user')
fig = px.bar(data_frame=df5, x='monetary_score', y='unique_user', height=300, width=500, color_discrete_sequence=['blue'], title='Monetary Score')
fig.show()

From score 1 until 5, the recency score almost has the same values.

# Conculution
1. FRM Segmentation: need to treat 'loyal customers' and 'potensial loyalists' as champion customers. The value of hibernating is very high, so we need to attract and promote it to customers to increase the frequency of transactions and monetary score.
2. The recency score and the monetary score have the same values, but the frequency score with 1 score is the biggest score. So we need to attend to increase the frequency score.